In [47]:
import networkx as nx
import plotly.graph_objs as go
from collections import Counter

def generate_unique_shape_graphs(k):
    """
    Generate all unique graph shapes with a total of vertices from 1 to k.
    
    Args:
    k (int): The maximum number of vertices.
    
    Returns:
    list: A list of unique graph shapes, each represented as a tuple of the number of vertices and a tuple of sorted edges.
    """
    
    all_graphs = {1: [nx.Graph()]}  # Base case: 1 vertex, 0 edges
    
    def get_graph_shape(G):
        degrees = sorted([d for _, d in G.degree()])
        return tuple(Counter(degrees).items())

    # Build graphs iteratively from 2 to k vertices
    for num_vertices in range(2, k + 1):
        new_graphs = []
        shapes = set()
        
        # For each existing graph with (num_vertices - 1)
        for existing_graph in all_graphs[num_vertices - 1]:
            # Create the new graph by adding a new vertex
            new_graph = existing_graph.copy()
            new_graph.add_node(num_vertices - 1)  # Add the new vertex (0-indexed)
            
            # Add the new graph to the list of new graphs (isolated vertex)
            shape = get_graph_shape(new_graph)
            if shape not in shapes:
                new_graphs.append(new_graph)
                shapes.add(shape)
            
            # Connect the new vertex to existing vertices in all possible unique ways
            for edges_to_add in range(1, num_vertices):
                graph_with_edges = new_graph.copy()
                for _ in range(edges_to_add):
                    # Add edge to the node with the least degree
                    degrees = dict(graph_with_edges.degree())
                    min_degree_node = min(range(num_vertices - 1), key=lambda x: degrees.get(x, 0))
                    graph_with_edges.add_edge(num_vertices - 1, min_degree_node)
                
                shape = get_graph_shape(graph_with_edges)
                if shape not in shapes:
                    new_graphs.append(graph_with_edges)
                    shapes.add(shape)

        all_graphs[num_vertices] = new_graphs
    
    # Format the result: convert graphs to tuples of (num_vertices, edges)
    result = []
    for num_vertices in range(1, k + 1):
        for g in all_graphs[num_vertices]:
            edges = tuple(sorted(g.edges()))
            result.append((num_vertices, edges))
    
    return result

def plot_graphs(graphs):
    """
    Plot all generated graphs using Plotly.
    
    Args:
    graphs (list): List of graphs to plot.
    """
    
    for index, (num_vertices, graph_edges) in enumerate(graphs):
        G = nx.Graph()
        G.add_nodes_from(range(num_vertices))
        G.add_edges_from(graph_edges)
        
        # Create a position layout
        if graph_edges:
            pos = nx.spring_layout(G)
        else:
            pos = {i: (i, 0) for i in range(num_vertices)}
        
        # Extracting node positions
        x_nodes = [pos[i][0] for i in G.nodes()]
        y_nodes = [pos[i][1] for i in G.nodes()]

        # Extracting edges for plotting
        edge_x = []
        edge_y = []
        for edge in G.edges():
            x_edge = [pos[edge[0]][0], pos[edge[1]][0], None]
            y_edge = [pos[edge[0]][1], pos[edge[1]][1], None]
            edge_x += x_edge
            edge_y += y_edge
        
        # Create Plotly traces for edges
        edge_trace = go.Scatter(
            x=edge_x, 
            y=edge_y, 
            line=dict(width=0.5, color='gray'),
            hoverinfo='none', 
            mode='lines'
        )
        
        # Create Plotly traces for nodes
        node_trace = go.Scatter(
            x=x_nodes, 
            y=y_nodes,
            mode='markers+text',
            text=[str(i) for i in G.nodes()],
            textposition="middle center",
            hoverinfo='text',
            marker=dict(
                showscale=False,
                size=20,
                color='red',
                line=dict(width=2, color='black')
            ),
            textfont=dict(
                size=16,
                color='white'
            )
        )
        
        # Create the figure
        fig = go.Figure(data=[edge_trace, node_trace],
                        layout=go.Layout(
                            title=f"Graph with {num_vertices} vertices, {len(graph_edges)} edges",
                            titlefont_size=16,
                            showlegend=False,
                            hovermode='closest',
                            margin=dict(b=0, l=0, r=0, t=40),
                            xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                            yaxis=dict(showgrid=False, zeroline=False, showticklabels=False))
                        )
        fig.show()

# Example usage:
k = 5  # Change this value to generate and plot up to k vertices
graphs = generate_unique_shape_graphs(k)
plot_graphs(graphs)

In [48]:
def remove_non_planar_graphs(graphs):
    """
    Remove non-planar graphs from the list of generated graphs.
    
    Args:
    graphs (list): A list of graph representations (num_vertices, edges).
    
    Returns:
    list: A list of only planar graphs.
    """
    planar_graphs = []
    
    for num_vertices, graph_edges in graphs:
        G = nx.Graph()
        G.add_nodes_from(range(num_vertices))
        G.add_edges_from(graph_edges)
        
        is_planar, _ = nx.check_planarity(G)
        if is_planar:
            planar_graphs.append((num_vertices, graph_edges))
    
    return planar_graphs

planar_graphs = remove_non_planar_graphs(graphs)

# Step 3: Plot the planar graphs
plot_graphs(planar_graphs)